# Assignment 1 - Sliding Problem
## <font color='red'>Checkpoint: </font>June 14, 11:59pm
## <font color='red'>Due Date: </font>June 17, 11:59pm

---

## 1. Defining 3x3 sliding problem environment
No modification should be done in this part.

In [1]:
import random
import time

class state:
    """ State of sliding number puzzle
        Contains array of values called 'board' to indicate
        tile positions, and the position of tile '0', which
        indicates the empty space on the board.         """
    
    boardSize=3

    def __init__(self,s=None):

        if s == None:
            
            tiles=range(self.boardSize*self.boardSize).__iter__()
            self.board=[[next(tiles) for i in range(self.boardSize)] for j in range(self.boardSize)]

            #keep track of empty position
            self.position=[0,0]
        
        else:
            #copy the board
            self.board=[]
            for row in s.board:
                self.board.append(list(row))

            #copy the positions    
            self.position=list(s.position)
            
        
    def __str__(self):
        rstr=''
        for row in self.board:
            rstr += str(row) + '\n'
        return rstr
    
    #overload to allow comparison of lists and states with ==
    def __eq__(self,other):
        if isinstance(other, state):
            return self.board == other.board
        else:
            return NotImplemented

## 2. Defining nodes for search graph
No modification should be done in this part.

In [2]:
class node:
    
    nodeCount=0
    
    def __init__(self, p, a, c, s):
        
        #keep track of how many nodes were created
        self.__class__.nodeCount += 1    
        self.nodeID=self.nodeCount
        
        self.parent=p
        self.cost=c
        self.action=a
        self.state=s
        
    #test equivalence Should be state

    def __str__(self):
        rstr= 'NodeID: ' + str(self.nodeID) + '\n'
        if self.parent != None:
            rstr+='Parent: ' + str(self.parent.nodeID) + '\n'
        if self.action != None:
            rstr+='Action: ' + self.action  + '\n'
        rstr+='Cost:   ' + str(self.cost) + '\n'
        rstr+='State:\n' + str(self.state)
        return rstr

In [3]:
def childNode(n, action, problem):
    return node(n,action, n.cost + 1, problem.apply(action,state(n.state)))

## 3. Defining problem
No modification should be done in this part.

Possible actions of our agents are 
* 'U' - Up
* 'L' - Left
* 'D' - Down
* 'R' - Right




In [4]:
#problem
class problem:
    """Class that defines a search problem"""

    def __init__(self):
        self.actions=['U','L','D','R']
        self.initialState=[[]]
        self.goalState=[[]]

    def apply(self,a,s):

        #positions after move, still refers to s.position object
        post=s.position

        #make a copy
        pre=list(post)
        
        #compute post position
        if a == 'U':
            post[0]=max(pre[0]-1,0)
        elif a == 'L':            
            post[1]=max(pre[1]-1,0)
        elif a == 'D':
            post[0]=min(pre[0]+1,s.boardSize-1)
        elif a == 'R':
            post[1]=min(pre[1]+1,s.boardSize-1)
        else:
            print('Undefined action: ' + str(a))
            raise StandardError('Action not defined for this problem!')

        #store the old tile
        tile=s.board[pre[0]][pre[1]]
        
        s.board[pre[0]][pre[1]]=s.board[post[0]][post[1]]
        s.board[post[0]][post[1]]=tile      

        return s
        
    def applicable(self,s):
        actionList=[]

        #check if actions are applicable
        #Not in top row
        if s.position[0]>0:
            actionList.append('U')

        #not in left most col
        if s.position[1]>0:
            actionList.append('L')

        #not in bottom row
        if s.position[0]<(s.boardSize-1):
            actionList.append('D')

        #not in right col
        if s.position[1]<(s.boardSize-1):
            actionList.append('R')

        return actionList

    def goalTest(self,s):
        return self.goalState==s     

In [5]:
def applyRndMoves(numMoves,s,p):
    for i in range(numMoves):
        p.apply(p.actions[random.randint(0,3)],s)

# Solving Sliding problem
1.   Breadth First Search (BFS) Approach
> *It will find the closest solution, by searching layer-by-layer.*

> The BFS explores an entire layer before progressing.  Each layer consists of a list of nodes.  These lists are looped through, and expanded into a new list based on the available directions to explore for that node at it's given state.  Once a list is exhausted, it is replaced by the new list, and is then expanded.  The cycle ends once 15 layers, or 15 lists have been explored, or when a solution is found.  

2.   Depth First Search (DFS) Approach
> *It will find the first solution, by searching as deep as possible first.*

> The DFS method explores as deeply as possible, before branching out to adjacent nodes.  This was accomplished using recurrence and a for loop.  The for loop would loop through all directions in a function, and call on the function to explore downwards in each one.  Once the function reached it's maximum depth of 15 without finding a solution, it would return "None," allowing the previously halted function above it to explore a new, adjacent path, or return "None" itself to allow the function above it to explore a new path, etc.  


In [36]:
class Searches:    
    
    def BFS(self, problem):
        #write your code here
        path_cost = 0 
        
        n = node(p = None, s=problem.initialState, c=path_cost, a=None) # the parent with initial_State.
                
        if problem.goalTest(n.state): # if we already have init state==goal_state
            return n
        
        frontier = []
        frontier.append(n) # append the parent 
        
        explored = []
        while frontier:
            n_ode = frontier.pop(0)
            
            if problem.goalTest(n_ode): # if we reach the solution then return it.
                return n_ode
            if n_ode.state not in explored:
                explored.append(n_ode.state)   
            
            actions = problem.applicable(n_ode.state) #actions that need to be performed on the node
            for a in actions:
                child = childNode(n_ode, a, problem) # creation of children.
                
                if child not in frontier or child.state not in explored:
                    if problem.goalTest(child.state):
                        return child
                    frontier.append(child)
                    #explored.put(child.state)
        return None
             
    def DFS(self, problem):
        path_cost = 0 
        parent = node(p = None, s=problem.initialState, c=path_cost, a=None)
        if problem.goalTest(parent.state): # if we already have init state==goal_state
            return parent
        explored = []
        return self.DFS_helper(problem, explored, parent)
        
    def DFS_helper(self, problem, explored, node):
        if problem.goalTest(node.state): # base case
            return node
        explored.append(node.state)       

        for a in problem.applicable(node.state):
            child = childNode(node, a, problem)
            
            if child.state not in explored:
                self.DFS_helper(problem, explored, child)
                
        
        return None
        

# Test your solution


In [39]:
if __name__ == '__main__':
       
    search = Searches()
    
    p = problem()
    s = state()
    
    p.goalState = state(s)
    
    p.apply('R',s)
    p.apply('R',s)
    p.apply('D',s)
    p.apply('D',s)
    p.apply('L',s)
    
    p.initialState=state(s) 
    print("Initial State \n", p.initialState)    
    
## Uncomment for testing BFS solution

    print('=== BFS  ===')
    startTime=time.process_time()
    res=search.BFS(p)
    print(res)
    print("Time " + str(time.process_time()-startTime))
    print("Explored Nodes: "+ str(node.nodeCount))
 
    
    
    print("Generating Random Position")
    si=state(s)
    applyRndMoves(15,si,p)
    p.initialState=si
    print(si)
    
    startTime=time.process_time()
    
    print('=== BFS  ===')
    startTime=time.process_time()
    res=search.BFS(p)
    print(res)
    print("Time " + str(time.process_time()-startTime))
    print("Explored Nodes: "+ str(node.nodeCount))

    print("\n")
    print("################[DFS]###################")

## Uncomment for testing DFS solution

    
    print(p.initialState)
    
    
    print('=== DFS  ===')
    startTime=time.process_time()
    res=search.DFS(p)
    print(res)
    print("Time " + str(time.process_time()-startTime))
    print("Explored Nodes: "+ str(node.nodeCount))
 
    
    
    print("Generating Random Position")
    si = state(s)
    applyRndMoves(15,si,p)
    p.initialState=si
    print(si)
    
    startTime = time.process_time()
    
    print('=== DFS  ===')
    startTime=time.process_time()
    res=search.DFS(p)
    print(res)
    print("Time " + str(time.process_time()-startTime))
    print("Explored Nodes: "+ str(node.nodeCount))
    

Initial State 
 [1, 2, 5]
[3, 4, 8]
[6, 0, 7]

=== BFS  ===
NodeID: 53176
Parent: 53016
Action: L
Cost:   5
State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Time 0.0048269999999988045
Explored Nodes: 53176
Generating Random Position
[0, 2, 5]
[1, 3, 4]
[6, 7, 8]

=== BFS  ===
NodeID: 53546
Parent: 53306
Action: L
Cost:   6
State:
[0, 1, 2]
[3, 4, 5]
[6, 7, 8]

Time 0.005651000000000295
Explored Nodes: 53546


################[DFS]###################
[0, 2, 5]
[1, 3, 4]
[6, 7, 8]

=== DFS  ===


RecursionError: maximum recursion depth exceeded in comparison

# Results
For BFS it took about 0.001ms to find a goal which 3*3 matrix. values in order.
However, DFS I used recursive method, it could not converge into optimal solution due to maximum recursive depth exceeded. Therefore, it's never converged to the goal. 

# Conclusions
After looking at both BFS AND DFS, I find BFS is much faster in our problem. 